# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [16]:
X = boston_features
y = pd.DataFrame(boston.target,columns=['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Fit the model and apply the model to the make test set predictions

In [19]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train,y_train)

y_pred_test = linreg.predict(X_test)
y_pred_train = linreg.predict(X_train)

Calculate the residuals and the mean squared error

In [20]:
test_residuals = y_pred_test - y_test
train_residuals = y_pred_train - y_train

MSE_test = np.mean(test_residuals**2)
MSE_train = np.mean(train_residuals**2)

print("MSE test: " + str(MSE_test))
print("MSE train: " + str(MSE_train))

MSE test: target    10.85627
dtype: float64
MSE train: target    18.063398
dtype: float64


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [34]:
def kfolds(data, k):
    # Force data as pandas dataframe
    df = pd.DataFrame(data)
    #list that will be returned
    k_folds = []
    
    #get data size
    length_of_df = len(df)
    #make a variable that stores the amount of folds that require one extra row
    needs_one_larger = length_of_df%k
    #make a variable that stores the fold size
    size_of_fold = length_of_df//k
    
    #variables to be used for indexing the dataframe
    row_start = 0
    row_end = size_of_fold
    
    for i in range(k):
        # add 1 to fold size to account for leftovers 
        if needs_one_larger > 0:
            row_end += 1
            needs_one_larger-=1
        
        #append dataframe section to k_folds list
        k_folds.append( df.iloc[row_start:row_end] )
        
        #set indices for next iteration of loop
        row_start = row_end
        row_end += size_of_fold
    
    
    return k_folds

### Apply it to the Boston Housing Data

In [44]:
# Make sure to concatenate the data again
data_all = pd.concat([y,X], axis=1)
len(data_all)

506

In [45]:
k_folded_data = kfolds(data_all, 5)

In [46]:
for i in range(len(k_folded_data)):
    print(len(k_folded_data[i]))


102
101
101
101
101


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [56]:
test_errs = []
train_errs = []
k=5

k_folded_data = kfolds(data_all, k)

for n in range(k):
    #get the particular fold from the folded data list
    fold = k_folded_data[n]
    
    # Split in train and test for the fold.
    # Training data is all of the data but with the fold rows removed.
    train = data_all.drop(fold.index.values)
    train_x = train.drop(['target'],axis=1)
    train_y = train['target']
    test = fold
    test_x = test.drop(['target'],axis=1)
    test_y = test['target']
    
    # Fit a linear regression model
    linreg.fit(train_x,train_y)
    
    #Evaluate Train and Test Errors
    y_pred_train = linreg.predict(train_x)
    y_pred_test = linreg.predict(test_x)
    
    train_residuals = y_pred_train - train_y
    test_residuals = y_pred_test - test_y
    
    train_MSE =  np.mean(train_residuals**2)
    test_MSE = np.mean(test_residuals**2)
    
    print("Fold: {}, Train MSE: {:0.2f}, Test MSE: {:0.2f}".format(n,train_MSE,test_MSE))

# print(train_errs)
# print(test_errs)


Fold: 0, Train MSE: 17.92, Test MSE: 13.02
Fold: 1, Train MSE: 17.36, Test MSE: 14.63
Fold: 2, Train MSE: 15.55, Test MSE: 24.81
Fold: 3, Train MSE: 11.04, Test MSE: 55.24
Fold: 4, Train MSE: 17.23, Test MSE: 19.02


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [62]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg,X,y,cv=5,scoring="neg_mean_squared_error")

-cv_5_results

array([13.0161921 , 14.62832183, 24.81432997, 55.24107773, 19.022338  ])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [64]:
np.mean(-cv_5_results)

25.344451926139975

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!